In [1]:
 
import gensim 
import transformers
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model



from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


/opt/anaconda3/envs/mothership/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
train, test = tfds.load('imdb_reviews', split=['train', 'test'], shuffle_files=True)
assert isinstance(train, tf.data.Dataset)

INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load dataset info from /Users/nargestabari/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
INFO:absl:Reusing dataset imdb_reviews (/Users/nargestabari/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split ['train', 'test'], from /Users/nargestabari/tensorflow_datasets/imdb_reviews/plain_text/1.0.0


In [3]:
print(len(train), len(test))
print(train.take(2))

train = tfds.as_dataframe(train)
test = tfds.as_dataframe(test)
print(train.head(2))
print(test.head(2))

25000 25000
<TakeDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>
   label                                               text
0      0  b"This was an absolutely terrible movie. Don't...
1      0  b'Mann photographs the Alberta Rocky Mountains...
   label                                               text
0      1  b"A blackly comic tale of a down-trodden pries...
1      1  b"There are films that make careers. For Georg...


In [4]:
X_train = train['text'].str.decode('utf-8')
X_test = test['text'].str.decode('utf-8')



tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)

X_tokenized_train = tokenizer.texts_to_sequences(X_train)
X_tokenized_test = tokenizer.texts_to_sequences(X_test)


In [5]:

max_review_len=500
X_train_padd = sequence.pad_sequences(X_tokenized_train, maxlen=max_review_len, padding='pre', truncating='pre')
X_test_padd = sequence.pad_sequences(X_tokenized_test, maxlen=max_review_len)


In [6]:
# The model: 

num_words=5000
model = Sequential()
model.add(Embedding(num_words, output_dim=50, input_length=max_review_len ))
model.add(Bidirectional(LSTM(units=124)))
model.add(Dropout(20))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

plot_model(model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 50)           250000    
_________________________________________________________________
bidirectional (Bidirectional (None, 248)               173600    
_________________________________________________________________
dropout (Dropout)            (None, 248)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 249       
Total params: 423,849
Trainable params: 423,849
Non-trainable params: 0
_________________________________________________________________
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [13]:
Y_train = train['label']
Y_test = test['label']

model.fit(X_train_padd, Y_train, epochs=3, batch_size=32 )

# print(m)

Epoch 1/3


ValueError: in user code:

    /opt/anaconda3/envs/mothership/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /opt/anaconda3/envs/mothership/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/envs/mothership/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/envs/mothership/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/envs/mothership/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/envs/moth